In [1]:
import $file.common
import common._
import cats.data._, cats._, cats.implicits._
import doobie.implicits._
import doobie.util.fragment._

import $file.$     

import common._

import cats.data._, cats._, cats.implicits._

import doobie.implicits._

import doobie.util.fragment._

# Variation 8a. Finally, QUEΛ

In [2]:
trait QUEΛ[Repr[_]]{
    // base types
    
    def bool(b: Boolean): Repr[Boolean]
    def int(i: Int): Repr[Int]
    def str(s: String): Repr[String]
    
    def >(i1: Repr[Int], i2: Repr[Int]): Repr[Boolean]
        
    // ADTs
    
    def tuple2[A, B](a: Repr[A], b: Repr[B]): Repr[(A, B)]
    
    def none[A]: Repr[Option[A]]
    def some[A](a: Repr[A]): Repr[Option[A]]
    def exists[A](o: Repr[Option[A]])(
        cond: Repr[A] => Repr[Boolean]): Repr[Boolean]
    
    // Comprehensions
    
    def bind[A, B](q: Repr[List[A]])(
        f: Repr[A] => Repr[List[B]]): Repr[List[B]]
    def pure[A](a: Repr[A]): Repr[List[A]]
    def where[A](cond: Repr[Boolean])(q: Repr[List[A]]): Repr[List[A]]
    
    // Equality
    
    def ===[A](a1: Repr[A], a2: Repr[A]): Repr[Boolean]
}

object QUEΛ{
    object syntax{
        implicit class ComprehensionOps[Repr[_], A](la: Repr[List[A]])(implicit Q: QUEΛ[Repr]){
            def flatMap[B](cont: Repr[A] => Repr[List[B]]): Repr[List[B]] = 
                Q.bind(la)(cont)
            def map[B](f: Repr[A] => Repr[B]): Repr[List[B]] = 
                Q.bind(la)(a => Q.pure(f(a)))
            def withFilter(p: Repr[A] => Repr[Boolean]): Repr[List[A]] = 
                Q.bind(la)(a => Q.where(p(a))(Q.pure(a)))
        }

        implicit class BinOps[Repr[_], A](a1: Repr[A])(implicit Q: QUEΛ[Repr]){
            def ===(a2: Repr[A]): Repr[Boolean] = Q.===(a1, a2)
            def >(a2: Repr[Int])(implicit eq: Repr[A] =:= Repr[Int]): Repr[Boolean] = 
                Q.>(eq(a1), a2)
        }

        implicit def Tuple2QUEΛ[Repr[_], A, B](t2: (Repr[A], Repr[B]))(implicit Q: QUEΛ[Repr]): Repr[(A, B)] = 
            Q.tuple2(t2._1, t2._2)
        
        implicit class OptionOps[Repr[_], A](o: Repr[Option[A]])(implicit Q: QUEΛ[Repr]){
            def exists(cond: Repr[A] => Repr[Boolean]): Repr[Boolean] = 
                Q.exists(o)(cond)
        }

        implicit def IntQUEΛ[Repr[_]](i: Int)(implicit Q: QUEΛ[Repr]): Repr[Int] = 
            Q.int(i)
    }
}

defined trait QUEΛ
defined object QUEΛ

Our usual case classes, but used as phantom types in the API.

In [3]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    district: String,
    population: Int)

case class World(
    countries: Map[String, Country], 
    cities: Map[Int, City])

defined class Country
defined class City
defined class World

In [4]:
trait CityModel[Repr[_]]{
    def id(city: Repr[City]): Repr[Int]
    def name(city: Repr[City]): Repr[String]
    def country(country: Repr[City]): Repr[String]
    def population(city: Repr[City]): Repr[Int]
}

trait CountryModel[Repr[_]]{
    def code(country: Repr[Country]): Repr[String]
    def name(country: Repr[Country]): Repr[String]
    def capital(country: Repr[Country]): Repr[Option[Int]]
}

trait WorldModel[Repr[_]]{
    val Ci: CityModel[Repr]
    val Co: CountryModel[Repr]
    def countries: Repr[List[Country]]
    def cities: Repr[List[City]]
}

object WorldModel{
    
    object syntax{
        def countries[Repr[_]](implicit W: WorldModel[Repr]) = 
            W.countries
        def cities[Repr[_]](implicit W: WorldModel[Repr]) = 
            W.cities
        def name[Repr[_]](c: Repr[Country])(implicit W: WorldModel[Repr]) = 
            W.Co.name(c)
        def capital[Repr[_]](c: Repr[Country])(implicit W: WorldModel[Repr]) = 
            W.Co.capital(c)
        def id[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.id(c)        
        def cityName[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.name(c)        
        def population[Repr[_]](c: Repr[City])(implicit W: WorldModel[Repr]) = 
            W.Ci.population(c)
    }
}

defined trait CityModel
defined trait CountryModel
defined trait WorldModel
defined object WorldModel

In [5]:
import QUEΛ.syntax._, WorldModel.syntax._

import QUEΛ.syntax._, WorldModel.syntax._

In [6]:
def largeCapitals[Repr[_]: QUEΛ: WorldModel]: Repr[List[(String, String)]] = 
    for {
        country <- countries
        city <- cities if capital(country).exists(_ === id(city))
        if population(city) > 8000000
    } yield (cityName(city), name(country))

defined function largeCapitals

In [6]:
/*def query[Repr[_]](implicit Q: QUEΛ[Repr], W: WorldModel[Repr]): Repr[List[(String, String)]] = 
    Q.bind(W.countries){ country => 
        Q.bind(W.cities){ city => 
            Q.where(Q.===(W.Co.capital(country), W.Ci.id(city))){
                Q.pure(Q.tuple2(W.Ci.name(city), W.Co.name(country)))
            }
        }
    }
  */  

In [7]:
class StdQUEΛ[F[_]: Monad, S] extends QUEΛ[ReaderT[F, S, ?]]{
    // BASE TYPES

    def bool(b: Boolean): ReaderT[F, S, Boolean] = 
        Kleisli.pure(b)
    def int(i: Int): ReaderT[F, S, Int] = 
        Kleisli.pure(i)
    def str(s: String): ReaderT[F, S, String] = 
        Kleisli.pure(s)

    def >(i1: ReaderT[F, S, Int], i2: ReaderT[F, S, Int]): ReaderT[F, S, Boolean] = 
        (i1, i2).mapN(_ > _)

    // ADTs

    def tuple2[A, B](a: ReaderT[F, S, A], b: ReaderT[F, S, B]): ReaderT[F, S, (A, B)] = 
        Applicative[ReaderT[F, S, ?]].tuple2(a,b)

    def none[A]: ReaderT[F, S, Option[A]] = 
        Kleisli.pure(None)
    def some[A](a: ReaderT[F, S, A]): ReaderT[F, S, Option[A]] = 
        a.map(Some(_))
    def exists[A](a: ReaderT[F, S, Option[A]])(
        cond: ReaderT[F, S, A] => ReaderT[F, S, Boolean]): ReaderT[F, S, Boolean] = 
        a.flatMap{ _.fold(Kleisli{ s: S => false.pure[F]}){
            a => cond(Kleisli.pure(a))
        }}

    // COMPREHENSIONS

    def bind[A, B](q: ReaderT[F, S, List[A]])(
            f: ReaderT[F, S, A] => ReaderT[F, S, List[B]]): ReaderT[F, S, List[B]] = 
        q.flatMap{ _.traverse((a: A) => 
            f(Kleisli.pure(a))).map(_.flatten)
        }
    def pure[A](a: ReaderT[F, S, A]): ReaderT[F, S, List[A]] = 
        a.map(List(_))
    def where[A](cond: ReaderT[F, S, Boolean])(q: ReaderT[F, S, List[A]]): ReaderT[F, S, List[A]] = 
        cond.flatMap{ if (_) q else Kleisli.pure(List()) }

    // EQUALITY

    def ===[A](a1: ReaderT[F, S, A], a2: ReaderT[F, S, A]): ReaderT[F, S, Boolean] = 
        (a1, a2).mapN(_ == _)
}

defined class StdQUEΛ

In [7]:
import $ivy.`com.softwaremill.common::tagging:2.2.1`
import com.softwaremill.tagging._

type WR[T] = Kleisli[Eval, World, T] @@ Int
val w: WR[String] = Kleisli.pure[Eval, World, String]("a").taggedWith[Int]
implicitly[WR[String] =:= Kleisli[Eval, World, String] @@ Int]

cmd7.sc:6: could not find implicit value for parameter e: Helper.this.WR[String] =:= cats.data.Kleisli[cats.Eval,cmd7.this.cmd2.World,String] @@ Int
val res7_4 = implicitly[WR[String] =:= Kleisli[Eval, World, String] @@ Int]
                       ^Compilation Failed

: 

In [ ]:
/* TBD
import $ivy.`com.softwaremill.common::tagging:2.2.1`
import com.softwaremill.tagging._
type WorldReader[T] = ReaderT[Eval, World, T] @@  WRC
*/

type WorldReader[T] = ReaderT[Eval, World, T]

object WorldReader{    
    
    implicit val QUEΛ = new StdQUEΛ[Eval, World]

    implicit object Model extends WorldModel[WorldReader]{
        val Ci = new CityModel[WorldReader]{
            def id(city: WorldReader[City]): WorldReader[Int] = 
                city.map{_.id}
            def name(city: WorldReader[City]): WorldReader[String] = 
                city.map{_.name}
            def country(city: WorldReader[City]): WorldReader[String] = 
                city.map{_.countryCode}
            def population(city: WorldReader[City]): WorldReader[Int] = 
                city.map{_.population}
        }

        val Co = new CountryModel[WorldReader]{
            def code(country: WorldReader[Country]): WorldReader[String] = 
                country.map{_.code}
            def name(country: WorldReader[Country]): WorldReader[String] = 
                country.map{_.name}
            def capital(country: WorldReader[Country]): WorldReader[Option[Int]] = 
                country.map{_.capital}
        }

        def countries: WorldReader[List[Country]] = 
            Kleisli{ world => Eval.now(world.countries.values.toList) }

        def cities: WorldReader[List[City]] = 
            Kleisli{ world => Eval.now(world.cities.values.toList) }
    }
}

In [ ]:
val realCountries: List[Country] = 
    sql"select code, name, capital from country"
        .query[Country].to[List].transact(xa).unsafeRunSync

val realCities: List[City] = 
    sql"select id, name, countryCode, district, population from city"
        .query[City].to[List].transact(xa).unsafeRunSync

val realMapCities: Map[Int, City] = 
    Map.from(realCities.map(city => (city.id, city)))

val realWorld = World(
    Map.from(realCountries.map(c => (c.code, c))), realMapCities)

In [ ]:
largeCapitals(WorldReader.QUEΛ, WorldReader.Model)
    .apply(realWorld)
    .value
    .timed

TBD: really bad performance, why? boxing?

TBD: Naive doobie instance.

TBD: Optimum SQL instance

TBD: Optimun MongoDB instance.